<a href="https://colab.research.google.com/github/tomonari-masada/course2025-nlp/blob/main/09_token_by_token_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキスト生成を試してみる

* 「コマ送り」で、テキストが生成される様子を見てみる。

## 準備

In [ ]:
import torch
torch.set_grad_enabled(False)

* https://huggingface.co/Qwen/Qwen3-0.6B

## LLMの読み込み

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"
model = AutoModelForCausalLM.from_pretrained(model_name, dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model.device

## LLMの構造

* forwardメソッドは以下の場所で調べられる。
  * https://github.com/huggingface/transformers/tree/main/src/transformers/models/qwen3

In [ ]:
model

## トークナイザ

In [ ]:
tokenizer

## 通常の方法でのテキスト生成

### トークンID列の作成
* トークナイザの`apply_chat_template`メソッドを使う。

In [ ]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
model_inputs

### テキストの生成

In [ ]:
generated_ids = model.generate(
    input_ids=model_inputs.input_ids,
    attention_mask=model_inputs.attention_mask,
    max_new_tokens=300
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]
output = tokenizer.decode(output_ids, skip_special_tokens=True)
print(output)

### 入力に埋め込みを使う場合

In [ ]:
model.get_input_embeddings()(model_inputs.input_ids)

In [ ]:
input_embeddings = model.get_input_embeddings()(model_inputs.input_ids)
outputs = model(
    inputs_embeds=input_embeddings,
    attention_mask=model_inputs.attention_mask,
)
outputs

* つまり、入力側の埋め込みに手を加えてからモデルに入力することもできるということ。

## Causal LLMの出力

### モデルへの入力を改めて作る
* すでに上で行ったことを再度行っているだけ。

In [ ]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

### 出力の確認
* モデルのforwardメソッドを使う。

In [ ]:
outputs = model(
    input_ids=model_inputs.input_ids,
    attention_mask=model_inputs.attention_mask,
)
outputs

In [ ]:
outputs.logits

In [ ]:
outputs.logits.shape

### logitが最大のサブワード

In [ ]:
tokenizer.decode(outputs.logits[0,-1,:].argmax(dim=-1), skip_special_tokens=False)

## テキスト生成のループ

* 以上の観察を踏まえて、自分でテキスト生成のループを書いてみる。

In [ ]:
answer = ""

input_ids = model_inputs.input_ids
for _ in range(100):
    outputs = model(
        input_ids=input_ids,
        attention_mask=None,
    )
    next_token = tokenizer.decode(outputs.logits[0,-1,:].argmax(dim=-1), skip_special_tokens=False)
    print(f"subword [{next_token}]")
    answer += next_token
    input_ids = torch.cat((input_ids, outputs.logits[:,-1:,:].argmax(dim=-1)), dim=-1)
    if next_token == tokenizer.eos_token:
        break

print(answer)